# Blog

> My blog!

In [1]:
#| default_exp blog

In [3]:
#| export
from fasthtml.common import *
from monsterui.all import *
from productivity_app.utils import *
from pathlib import Path
import glob
from fastcore.ansi import ansi2html

import yaml
from execnb.nbio import read_nb
from execnb.shell import render_outputs

In [4]:
#| export
ar = APIRouter(prefix="/blog", body_wrap=layout)

In [12]:
#| export
path = Path()
if IN_NOTEBOOK: path = Path('../')

In [119]:
#| export
def get_notebooks():
    notebooks = []
    for nb in (path/'posts').rglob('*.ipynb'):
        if not nb.name.startswith('_') and '.ipynb_checkpoints' not in str(nb):
            notebooks.append(str(nb))
    return notebooks

In [142]:
#| export
def get_meta(nb): return yaml.safe_load(nb.cells[0].source.split('---')[1])

In [121]:
#| export
def get_nb_lang(nb): return nb['metadata']['kernelspec']['language']

In [122]:
#| export
from IPython.utils.text import strip_ansi
from html import escape
def _pre(s, xtra=''): return f"<pre {xtra}><code>{escape(s)}</code></pre>"
def _strip(s): return strip_ansi(escape(s))

In [123]:
#| export
def render_code_output(cell, lang='python', pygments=False, wrapper=Footer):
    if not cell.outputs: return ''
    
    def render_output(out):
        otype = out['output_type']
        if otype == 'stream':
            txt = ansi2html(''.join(out["text"]))
            xtra = '' if out['name']=='stdout' else "class='stderr'"
            is_err = '<span class' in txt
            return Safe(f"<pre {xtra}><code class='{'nohighlight hljs' if is_err else ''}'>{txt}</code></pre>")
        elif otype in ('display_data','execute_result'):
            data = out['data']
            _g = lambda t: ''.join(data[t]) if t in data else None
            if d := _g('text/html'): return Safe(apply_classes(d))
            if d := _g('application/javascript'): return Safe(f'<script>{d}</script>')
            if d := _g('text/markdown'): return render_md(d)
            if d := _g('text/latex'): return Safe(f'<div class="math">${d}$</div>')
            if d := _g('image/jpeg'): return Safe(f'<img src="data:image/jpeg;base64,{d}"/>')
            if d := _g('image/png'): return Safe(f'<img src="data:image/png;base64,{d}"/>')
            if d := _g('text/plain'): return escape(d)
            if d := _g('image/svg+xml'): return Safe(d)
        return ''
    
    res = Div(*map(render_output, cell.outputs))
    if res: return wrapper(res)

In [124]:
#| export
def render_nb(nb):
    "Render a notebook as a list of html elements"
    res = []
    lang = get_nb_lang(nb)
    for cell in nb.cells[1:]:  # Skip first cell which contains metadata
        if cell['cell_type']=='code':
            _output = render_code_output(cell)
            res.append(render_md(f'''\n```{lang}\n{cell.source}\n```\n'''))
            res.append(Card(_output) if _output else '')
        elif cell['cell_type']=='markdown':
            res.append(render_md(cell.source))
    return res

In [171]:
nb = read_nb('../posts/BasicTesting.ipynb')

In [184]:
nb.cells[26].outputs[0].data

```json
{ 'text/html': [ '<div>\n',
                 '<style scoped>\n',
                 '    .dataframe tbody tr th:only-of-type {\n',
                 '        vertical-align: middle;\n',
                 '    }\n',
                 '\n',
                 '    .dataframe tbody tr th {\n',
                 '        vertical-align: top;\n',
                 '    }\n',
                 '\n',
                 '    .dataframe thead th {\n',
                 '        text-align: right;\n',
                 '    }\n',
                 '</style>\n',
                 '<table border="1" class="dataframe">\n',
                 '  <thead>\n',
                 '    <tr style="text-align: right;">\n',
                 '      <th></th>\n',
                 '      <th>open_date</th>\n',
                 '      <th>ticker</th>\n',
                 '      <th>action</th>\n',
                 '    </tr>\n',
                 '  </thead>\n',
                 '  <tbody>\n',
                 '    <tr>\n',
                 '      <th>0</th>\n',
                 '      <td>2017-02-14</td>\n',
                 '      <td>A</td>\n',
                 '      <td>Buy</td>\n',
                 '    </tr>\n',
                 '    <tr>\n',
                 '      <th>2</th>\n',
                 '      <td>2017-02-16</td>\n',
                 '      <td>A</td>\n',
                 '      <td>Buy</td>\n',
                 '    </tr>\n',
                 '    <tr>\n',
                 '      <th>11</th>\n',
                 '      <td>2017-03-02</td>\n',
                 '      <td>A</td>\n',
                 '      <td>Buy</td>\n',
                 '    </tr>\n',
                 '    <tr>\n',
                 '      <th>13</th>\n',
                 '      <td>2017-03-04</td>\n',
                 '      <td>A</td>\n',
                 '      <td>Buy</td>\n',
                 '    </tr>\n',
                 '    <tr>\n',
                 '      <th>14</th>\n',
                 '      <td>2017-03-07</td>\n',
                 '      <td>A</td>\n',
                 '      <td>Buy</td>\n',
                 '    </tr>\n',
                 '  </tbody>\n',
                 '</table>\n',
                 '</div>'],
  'text/plain': [ '    open_date ticker action\n',
                  '0  2017-02-14      A    Buy\n',
                  '2  2017-02-16      A    Buy\n',
                  '11 2017-03-02      A    Buy\n',
                  '13 2017-03-04      A    Buy\n',
                  '14 2017-03-07      A    Buy']}
```

In [147]:
#| export
@ar
def index(): 
    metas = []
    for fpath in get_notebooks():
        _meta = get_meta(read_nb(fpath))
        _meta['fpath'] = fpath
        metas.append(_meta)
    metas.sort(key=lambda x: x['date'], reverse=True)
    return Div(
        Div(H1("My Blog", cls="mb-2"),
            Subtitle("Thoughts, tutorials and technical writings", cls=TextT.gray+TextT.lg),
            cls="text-center py-8"),
        Div(Grid(*map(blog_card, metas), cols=1),
            cls="max-w-4xl mx-auto px-4"))

In [ ]:
#| export
@ar
def blog_post(fpath:str): return render_nb(read_nb(fpath))

In [107]:
#| export
def blog_card(meta):
    def Tags(cats): return DivLAligned(map(Label, cats))
    
    return Card(
        DivLAligned(
            A(Img(src=meta.get('image',''), style="width:200px"), href=blog_post.to(fpath=meta['fpath'])),
            Div(cls='space-y-3 uk-width-expand')(
                H4(meta['title']),
                P(meta['description']),
                DivFullySpaced(map(Small, [meta['author'], meta['date']]), cls=TextT.muted),
                DivFullySpaced(
                    Tags(meta['categories']),
                    A("Read", cls=('uk-button', ButtonT.primary,'h-6'), href=blog_post.to(fpath=meta['fpath']))))),
        cls=CardT.hover)